# Test CRAWDAD User

TJ Kim

Date: 4/20/21

#### Summary:
Import and run Crawdad users.

In [46]:
# Import Generic Classes
import numpy as np
import copy
import random

# Import All Custom Classes
import os, sys
sys.path.append(os.path.pardir+"/solvers")
sys.path.append(os.path.pardir+"/one_mobility")
sys.path.append(os.path.pardir+"/crawdad_mobility")


from Server import *
from User import *
from Link import *
from Job import *
from get_one_sim_usr import *
from load_crawdad_usr import *

# Import Solver Classes
from Optim_PlanGenerator import *
from SeqGreedy_PlanGenerator import*
from Migration_Plans import *

In [2]:
# Load All 12 users and trim their info
db_names = []
for i in range(1,13):
    db_names += ['LifeMap_GS'+str(i)+'.db']
    
db = []
for name in db_names:
    name = '/home/tkim/CMU/MEC_research/crawdad_mobility/' + name
    df,trace = load_usr(name)
    db += [trace]

In [47]:
"""
Make Simulation Parameters
"""
sim_param = Sim_Params(time_steps=15, x_length = 5, y_length = 5, max_edge_length=3,num_path_limit=10)
boundaries = np.array([[0,sim_param.x_length],[0,sim_param.y_length]])

"""
Make Servers
"""

# Server Settings
num_server_l1 = 2
num_server_l2 = 6
num_server_l3 = 1

num_resource = 3
weak_range = np.array([[4,8],[1000,1500],[4,16]])
strong_range = np.array([[50,100],[100000,150000],[300,600]])

rsrc_cost = np.array([0.03, 0.01, 0.05])

rsrc_cost_scale_lv1 = 2
rsrc_cost_scale_lv2 = 1
rsrc_cost_scale_lv3 = 0.7

# Generate Server
servers_l1 = []
servers_l2 = []
servers_l3 = []
idx_counter = 0

for i in range(num_server_l1):
    servers_l1.append(Server(boundaries,level=1,rand_locs=True,locs=None))
    servers_l1[-1].server_resources(num_resource, weak_range, strong_range)
    servers_l1[-1].assign_id(idx_counter)
    servers_l1[-1].server_resources_cost(num_resource,rsrc_cost*rsrc_cost_scale_lv1)
    idx_counter += 1

for i in range(num_server_l2):
    servers_l2.append(Server(boundaries,level=2,rand_locs=True,locs=None))
    servers_l2[-1].server_resources(num_resource, weak_range, strong_range)
    servers_l2[-1].assign_id(idx_counter)
    servers_l2[-1].server_resources_cost(num_resource,rsrc_cost*rsrc_cost_scale_lv2)
    idx_counter += 1

for i in range(num_server_l3):
    servers_l3.append(Server(boundaries,level=3,rand_locs=False,locs=np.array([200,200])))
    servers_l3[-1].server_resources(num_resource, weak_range, strong_range)
    servers_l3[-1].assign_id(idx_counter)
    servers_l3[-1].server_resources_cost(num_resource,rsrc_cost*rsrc_cost_scale_lv3)
    idx_counter += 1

servers = servers_l1 + servers_l2 + servers_l3


"""
Make Links
"""

# Link Settings
num_link = [0,1,2,3]
prob_link = [0.5,0.2,0.2,0.1]
lv_minmax = np.array(([[500,1000],[10000,20000],[30000,50000]]))
lv1_transmission = 1
link_costs = np.array([0.05, 0.02, 0.01])
latency_settings = [10, 1] #[ms per switch, ms per mile]

links = Link(servers, num_link, prob_link, lv_minmax, link_costs, latency_settings,lv1_transmission)


In [48]:
def avg_speed(new_mvmt_array):
    """
    returns a single double mean speed that a user goes 
    given x,y inputs from new_mvmt_array
    """

    length = new_mvmt_array.shape[0]-1
    mean_mvmt = 0

    for i in range(length):
        x1,y1 = new_mvmt_array[i,0], new_mvmt_array[i,1]
        x2,y2 = new_mvmt_array[i+1,0], new_mvmt_array[i+1,1]
        dist = np.sqrt((x2-x1)**2 + (y2-y1)**2)

        mean_mvmt += dist/length

    return mean_mvmt

def draw_ts(mvmt_array, time_steps):
    """
    select subset of array of mvmt for x timesteps,
    adjust initial ts to be 0 (or just have x,y)
    """

    # select random TS
    total_run = mvmt_array.shape[0]
    init_ts = np.random.randint(0,int(total_run-time_steps))

    new_mvmt_array = mvmt_array[init_ts:init_ts+int(time_steps), 1:]
    return new_mvmt_array

In [49]:
"""
Make Users 
- First draw samples from dataste - 
"""

# User Settings
num_user = 100

max_speed = 80
num_path = 5
num_path_orig = 1
mvmt_class = 0

spd_thresh = [0.2, 5]
boundary_thresh = [boundaries[0,1],boundaries[1,1]]

# Generate Server
users = []
idx_counter = 0
infos = np.zeros([num_user, 5]) # <min x, max x, min y, max y, meanspd>

for i in range(num_user):
    
    confirmed = False
    
    # Add filtering based on mean loc and start location randomization
    while confirmed is False:
        db_idx = np.random.randint(len(db))
        mvmt_array = random.choice(list(db[db_idx].values()))
        if mvmt_array.shape[0] > sim_param.time_steps:
            # Take mean speed here:
            new_mvmt_array = draw_ts(mvmt_array, sim_param.time_steps)
            mean_spd = avg_speed(new_mvmt_array)
            xmin = min(new_mvmt_array[:,0])
            xmax = max(new_mvmt_array[:,0])
            ymin = min(new_mvmt_array[:,1])
            ymax = max(new_mvmt_array[:,1])
            
            if mean_spd >= spd_thresh[0] and mean_spd <= spd_thresh[1]:
                if (abs(xmax-xmin) <= boundary_thresh[0]) and (abs(ymax-ymin) <= boundary_thresh[1]):
                    confirmed = True
                
    # Edit initialization point of travel for more even distribution
    # Assuming all boundaries go from 0-x, 0-y
    xlow = -1*min(new_mvmt_array[:,0])
    xhigh = (boundary_thresh[0] - max(new_mvmt_array[:,0]))
    ylow = -1*min(new_mvmt_array[:,1])
    yhigh = (boundary_thresh[1] - max(new_mvmt_array[:,1]))
    x_offset = np.random.uniform(low= xlow, high=xhigh)
    y_offset = np.random.uniform(low=ylow,high=yhigh)
    
    new_mvmt_array[:,0] += x_offset
    new_mvmt_array[:,1] += y_offset
    
    xmin = min(new_mvmt_array[:,0])
    xmax = max(new_mvmt_array[:,0])
    ymin = min(new_mvmt_array[:,1])
    ymax = max(new_mvmt_array[:,1])
    infos[i] = np.array([xmin,xmax,ymin,ymax,mean_spd])
        
    users += [Crawdad_User(boundaries, sim_param.time_steps, max_speed, num_path, 
                           num_path_orig, new_mvmt_array, mean_spd, mvmt_class)]
    users[-1].generate_MC(servers)
    users[-1].assign_id(idx_counter)
    idx_counter += 1

In [50]:
np.set_printoptions(formatter={'float': lambda x: "{0:0.2f}".format(x)})
print(infos)

[[0.85 3.26 2.66 4.45 0.42]
 [1.90 4.33 0.04 2.38 0.26]
 [1.75 3.20 0.75 3.73 0.25]
 [0.04 0.56 2.53 4.67 0.20]
 [1.27 4.06 2.45 4.82 0.27]
 [1.11 2.68 0.76 3.24 0.21]
 [0.98 3.80 0.20 4.91 0.52]
 [0.55 4.97 1.33 5.00 1.34]
 [3.82 4.47 2.67 4.58 0.21]
 [0.61 4.96 2.34 3.79 0.45]
 [1.03 2.07 0.29 3.17 0.23]
 [2.77 4.88 1.46 3.19 0.29]
 [1.40 2.10 0.60 4.73 0.30]
 [2.44 4.99 2.45 3.14 0.36]
 [1.57 4.64 0.58 4.52 0.37]
 [1.65 4.66 1.57 4.09 0.28]
 [0.34 1.10 1.00 4.54 0.28]
 [4.31 4.89 1.80 4.65 0.21]
 [1.47 4.68 0.27 3.23 0.31]
 [3.69 4.47 1.40 4.65 0.29]
 [0.36 4.69 0.02 4.41 1.13]
 [2.13 4.72 0.48 3.62 0.31]
 [3.07 3.99 0.89 4.14 0.24]
 [4.18 4.51 0.34 3.60 0.24]
 [3.07 4.09 1.19 4.87 0.32]
 [1.38 4.15 0.89 4.16 0.31]
 [0.02 4.52 1.46 4.27 0.70]
 [1.31 4.87 1.26 2.98 0.72]
 [0.06 4.88 0.61 4.30 0.53]
 [2.90 4.45 0.60 4.53 0.33]
 [0.12 4.81 0.09 4.51 0.51]
 [0.61 3.56 0.06 4.20 0.37]
 [2.77 3.67 0.57 2.95 0.31]
 [2.21 2.83 1.62 4.69 0.22]
 [1.44 3.28 2.59 4.99 0.34]
 [0.89 4.41 1.00 2.5

In [19]:
new_mvmt_array

In [20]:
sim_param.time_steps

15